In [2]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [4]:
target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])

In [5]:
# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

In [7]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9555555555555556


In [8]:
print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

[0]
[0]


In [9]:
# create a random generator
import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x



In [10]:
for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[7.679623025527233, 3.2046909287818264, 5.6477860554003305, 1.0257578123431321]
[0]
[1]
[7.249149067912652, 2.311522827009492, 6.4326248154974, 2.2606221519601717]
[0]
[0]
[5.4981795106862945, 4.375557923565067, 2.322309221855858, 2.2191282437676993]
[0]
[0]
[7.690828715810722, 2.9307455777006735, 3.9558073773268307, 1.7001433629166358]
[1]
[1]
[7.492599104437831, 3.4730846542779537, 4.540614361256684, 0.890672980211667]
[1]
[1]


In [11]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)

In [12]:
bb.classify_example([3, 5, 4, 2])

0

In [13]:
from sygus_if import SyGuS_IF
X = [
    [1,2,1,1],
    [-1.1,1,1,42]
]
y = [
    1,
    0
]

In [18]:
sgf = SyGuS_IF()
sgf.fit(X,y)
# sgf.fit(X_train,y_train)
print(sgf.synthesized_function)

(define-fun func ((sepal_length Real) (sepal_width Real) (petal_length Real) (petal_width Real)) Real (ite (<= petal_length sepal_width) 0 (ite (<= sepal_width (* petal_width petal_width)) (ite (<= sepal_length (+ petal_length petal_width)) (ite (<= sepal_width (- sepal_length sepal_width)) 0 (ite (<= sepal_length (+ petal_length 1)) 0 1)) 1) (ite (<= sepal_length (+ petal_length petal_width)) (ite (= sepal_length (+ petal_length petal_width)) 1 (ite (<= sepal_width (+ petal_width petal_width)) (ite (= sepal_width (/ petal_length petal_width)) 1 (ite (<= sepal_length (+ petal_length 1)) 1 (ite (<= sepal_width (/ sepal_length sepal_width)) 1 0))) 1)) 1))))


In [17]:
y_pred_test = sgf.predict(X_test, y_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict(X_train, y_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.9555555555555556
Accuracy: 1.0
